# 10-714 Homework 2

In this homework, you will be implementing a neural network library in the needle framework. Reminder: __you must save a copy in drive__.

In [ ]:
# Code to set up the assignment
from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/MyDrive/
!mkdir -p 10714
%cd /content/drive/MyDrive/10714
!git clone https://github.com/dlsys10714/hw2.git
%cd /content/drive/MyDrive/10714/hw2

!pip3 install --upgrade --no-deps git+https://github.com/dlsys10714/mugrade.git

This homework builds off of Homework 1. First, in your Homework 2 directory, go to the files `autograd.py`, `ops.py` and `numpy_backend.py` in the `python/needle` directory, and fill in the code between `### BEGIN YOUR SOLUTION` and `### END YOUR SOLUTION` with your solutions from Homework 1. 

__A note__: When copying over your solutions from the previous `numpy_backend.py`, you should add something like `.astype(inputs[0].dtype)` to your methods' return statements to ensure that your methods don't change the dtype (in fact, you can directly use the previous snippet in all cases). This is because functions like `np.divide` will change their output type in order to present a more accurate answer. However, this may result in type conflicts in our current version of needle. Forcibly casting like this is not the optimal solution, and we will probably revisit it later, but it is an acceptable workaround for now.

In [1]:
import sys
sys.path.append('./python')
sys.path.append('./apps')

## Question 0 [5 points]

Before you begin implementing your Needle neural network library, you first have to implement a couple more ops. These are crucial ops for the loss function and optimizers that you will quickly find yourself needing below.

### PowerScalar

We will generally find it crucial to find the elementwise power (to a scalar) of a tensor, for example, to compute the variance in BatchNorm. You will also need to implement the `__pow__` method in `Tensor` for this op.

In [10]:
!python3 -m pytest -v -k "op_power_scalar"

============================= test session starts ==============================
platform linux -- Python 3.7.5, pytest-6.2.5, py-1.10.0, pluggy-1.0.0 -- /home/bowenc/dev/cmu/dlsys/hw2/hw2-env/bin/python3
cachedir: .pytest_cache
rootdir: /home/bowenc/dev/cmu/dlsys/hw2
collected 98 items / 95 deselected / 3 selected                                

tests/test_nn_and_optim.py::test_op_power_scalar_forward_1 PASSED        [ 33%]
tests/test_nn_and_optim.py::test_op_power_scalar_forward_2 PASSED        [ 66%]
tests/test_nn_and_optim.py::test_op_power_scalar_backward_1 PASSED       [100%]

=============================== warnings summary ===============================
hw2-env/lib/python3.7/site-packages/mugrade/mugrade.py:71
  /home/bowenc/dev/cmu/dlsys/hw2/hw2-env/lib/python3.7/site-packages/mugrade/mugrade.py:71: PytestUnknownMarkWarning: Unknown pytest.mark.hookwrapper - is this a typo?  You can register custom marks to avoid this warning - for details, see https://docs.pytest.org/en/sta

In [12]:
!python3 -m mugrade submit 'eAzORHnc5JIEkr0dqEig' -k "op_power_scalar"

submit
============================= test session starts ==============================
platform linux -- Python 3.7.5, pytest-6.2.5, py-1.10.0, pluggy-1.0.0
rootdir: /home/bowenc/dev/cmu/dlsys/hw2
collected 22 items / 21 deselected / 1 selected                                

tests/test_nn_and_optim.py 
Submitting op_power_scalar...
Grader test 1 passed
Grader test 2 passed
Grader test 3 passed
.

======================= 1 passed, 21 deselected in 2.53s =======================


### LogSoftmax

LogSoftmax is useful for implementing softmax loss since we can write its forward and backward pass in terms of it. It is simply the log of the softmax function:

$$
\mathsf{LogSoftmax}(x) = \log\left(\frac{\exp(x)}{\sum_i \exp(x_i)}\right)
$$

Consider how you could simplify this expression.

In [46]:
!python3 -m pytest -v -k "op_logsoftmax"

============================= test session starts ==============================
platform linux -- Python 3.7.5, pytest-6.2.5, py-1.10.0, pluggy-1.0.0 -- /home/bowenc/dev/cmu/dlsys/hw2/hw2-env/bin/python3
cachedir: .pytest_cache
rootdir: /home/bowenc/dev/cmu/dlsys/hw2
collected 98 items / 95 deselected / 3 selected                                

tests/test_nn_and_optim.py::test_op_logsoftmax_forward_1 PASSED          [ 33%]
tests/test_nn_and_optim.py::test_op_logsoftmax_stable_forward_1 PASSED   [ 66%]
tests/test_nn_and_optim.py::test_op_logsoftmax_backward_1 PASSED         [100%]

=============================== warnings summary ===============================
hw2-env/lib/python3.7/site-packages/mugrade/mugrade.py:71
  /home/bowenc/dev/cmu/dlsys/hw2/hw2-env/lib/python3.7/site-packages/mugrade/mugrade.py:71: PytestUnknownMarkWarning: Unknown pytest.mark.hookwrapper - is this a typo?  You can register custom marks to avoid this warning - for details, see https://docs.pytest.org/en/sta

In [4]:
!python3 -m mugrade submit 'eAzORHnc5JIEkr0dqEig' -k "op_logsoftmax"

submit
============================= test session starts ==============================
platform linux -- Python 3.7.5, pytest-6.2.5, py-1.10.0, pluggy-1.0.0
rootdir: /home/bowenc/dev/cmu/dlsys/hw2
collected 22 items / 21 deselected / 1 selected                                

tests/test_nn_and_optim.py 
Submitting op_logsoftmax...
Grader test 1 passed
Grader test 2 passed
Grader test 3 passed
Grader test 4 passed
Grader test 5 passed
.

=============================== warnings summary ===============================
tests/test_nn_and_optim.py::submit_op_logsoftmax
  ./python/needle/numpy_backend.py:177: RuntimeWarning: overflow encountered in exp
    exp_x = np.exp(inputs[0])

-- Docs: https://docs.pytest.org/en/stable/warnings.html
================= 1 passed, 21 deselected, 1 warning in 3.73s ==================


## Question 1 [5 points]

Before we can implement certain neural network library modules, we need an assortment
of methods to initialize tensors from various distributions.
In this question, you will implement different methods for weight initialization. Specifically, in `python/needle/init.py` implement the functions:
___
### Uniform
`needle.init.uniform(x, low=0.0, high=1.0)`

Fills the input Tensor with values drawn from the uniform distribution $\mathcal{U}(a,b)$.

##### Parameters
- `x` - Tensor
- `low` - lower bound of the uniform distribution
- `high` - upper bound of the uniform distribution
___

### Normal
`needle.init.normal(x, mean=0.0, std=1.0)`

Fills the input Tensor with values drawn from the normal distribution $\mathcal{N}(\text{mean},\text{std}^2)$.

##### Parameters
- `x` - Tensor
- `mean` - mean of the normal distribution
- `std` - standard deviation of the normal distribution
___

### Constant
`needle.init.constant(x, c=0.0)`

Fills the input Tensor with value `c`.

##### Parameters
- `x` - Tensor
- `c` - the value to fill the Tensor with
___

### Ones
`needle.init.ones(x, c=0.0)`

Fills the input Tensor with scalar value 1.

##### Parameters
- `x` - Tensor
___

### Zeros
`needle.init.zeros(x)`

Fills the input Tensor with scalar value 0.

##### Parameters
- `x` - Tensor
___

### Xavier uniform
`needle.init.xavier_uniform(x, gain=1.0)`

Fills the input Tensor with values according to the method described in [Understanding the difficulty of training deep feedforward neural networks](https://proceedings.mlr.press/v9/glorot10a/glorot10a.pdf), using a uniform distribution. The resulting Tensor will have values sampled from $\mathcal{U}(-a, a)$ where 
\begin{equation}
a = \text{gain} \times \sqrt{\frac{6}{\text{fan_in} + \text{fan_out}}}
\end{equation}

##### Parameters
- `x` - Tensor
- `gain` - optional scaling factor
___

### Xavier normal
`needle.init.xavier_normal(x, gain=1.0)`

Fills the input Tensor with values according to the method described in [Understanding the difficulty of training deep feedforward neural networks](https://proceedings.mlr.press/v9/glorot10a/glorot10a.pdf), using a normal distribution. The resulting Tensor will have values sampled from $\mathcal{N}(0, \text{std}^2)$ where 
\begin{equation}
\text{std} = \text{gain} \times \sqrt{\frac{2}{\text{fan_in} + \text{fan_out}}}
\end{equation}

##### Parameters
- `x` - Tensor
- `gain` - optional scaling factor
___

### Kaiming uniform
`needle.init.kaiming_uniform(x, mode='fan_in', nonlinearity='relu')`

Fills the input Tensor with values according to the method described in [Delving deep into rectifiers: Surpassing human-level performance on ImageNet classification](https://arxiv.org/pdf/1502.01852.pdf), using a uniform distribution. The resulting Tensor will have values sampled from $\mathcal{U}(-\text{bound}, \text{bound})$ where 
\begin{equation}
\text{bound} = \text{gain} \times \sqrt{\frac{3}{\text{fan_mode}}}
\end{equation}

Use the recommended gain value for ReLU: $\text{gain}=\sqrt{2}$.

##### Parameters
- `x` - Tensor
- `mode` - either `fan_in` or `fan_out`. Choosing `fan_in` preserves the magnitude of the variance of the weights in the forward pass. Choosing `fan_out` preserves the magnitudes in the backwards pass. 
- `nonlinearity` - the non-linear function
___

### Kaiming normal
`needle.init.kaiming_normal(x, mode='fan_in', nonlinearity='relu')`

Fills the input Tensor with values according to the method described in [Delving deep into rectifiers: Surpassing human-level performance on ImageNet classification](https://arxiv.org/pdf/1502.01852.pdf), using a uniform distribution. The resulting Tensor will have values sampled from $\mathcal{N}(0, \text{std}^2)$ where 
\begin{equation}
\text{std} = \frac{\text{gain}}{\sqrt{\text{fan_mode}}}
\end{equation}

Use the recommended gain value for ReLU: $\text{gain}=\sqrt{2}$.

##### Parameters
- `x` - Tensor
- `mode` - either `fan_in` or `fan_out`. Choosing `fan_in` preserves the magnitude of the variance of the weights in the forward pass. Choosing `fan_out` preserves the magnitudes in the backwards pass. 
- `nonlinearity` - the non-linear function

In [30]:
!python3 -m pytest -v -k "test_init"

============================= test session starts ==============================
platform linux -- Python 3.7.5, pytest-6.2.5, py-1.10.0, pluggy-1.0.0 -- /home/bowenc/dev/cmu/dlsys/hw2/hw2-env/bin/python3
cachedir: .pytest_cache
rootdir: /home/bowenc/dev/cmu/dlsys/hw2
collected 98 items / 87 deselected / 11 selected                               

tests/test_nn_and_optim.py::test_init_uniform_1 PASSED                   [  9%]
tests/test_nn_and_optim.py::test_init_normal_1 PASSED                    [ 18%]
tests/test_nn_and_optim.py::test_init_constant_1 PASSED                  [ 27%]
tests/test_nn_and_optim.py::test_init_ones_1 PASSED                      [ 36%]
tests/test_nn_and_optim.py::test_init_zeros_1 PASSED                     [ 45%]
tests/test_nn_and_optim.py::test_init_kaiming_uniform_1 PASSED           [ 54%]
tests/test_nn_and_optim.py::test_init_kaiming_uniform_2 PASSED           [ 63%]
tests/test_nn_and_optim.py::test_init_kaiming_normal_1 PASSED            [ 72%]
tests/test

In [29]:
!python3 -m mugrade submit 'eAzORHnc5JIEkr0dqEig' -k "init"

submit
============================= test session starts ==============================
platform linux -- Python 3.7.5, pytest-6.2.5, py-1.10.0, pluggy-1.0.0
rootdir: /home/bowenc/dev/cmu/dlsys/hw2
collected 22 items / 21 deselected / 1 selected                                

tests/test_nn_and_optim.py 
Submitting init...
Grader test 1 passed
Grader test 2 passed
Grader test 3 passed
Grader test 4 passed
Grader test 5 passed
Grader test 6 passed
.

======================= 1 passed, 21 deselected in 4.54s =======================


## Question 2 [30 points]

In this question, you will implement additional modules in `python/needle/nn.py`. Specifically, for the following modules described below, initialize any variables of the module in the constructor, and fill out the `forward` method. 
___

### Linear
`needle.nn.Linear(in_features, out_features, bias=True, device=None, dtype="float32")`

Applies a linear transformation to the incoming data: $y = xA^T + b$. The input shape is $(N, *, H_{in})$ where * means any number of additional dimensions and $H_{in}=\text{in_features}$. The output shape is $(N, *, H_{out})$ where all but the last dimension are the same shape as the input and $H_{out}=\text{out_features}$.

Be careful to explicitly broadcast the bias term to the correct shape -- Needle does not support implicit broadcasting.


##### Parameters
- `in_features` - size of each input sample
- `out_features` - size of each output sample
- `bias` - If set to `False`, the layer will not learn an additive bias.

##### Variables
- `weight` - the learnable weights of shape (`in_features`, `out_features`). The values are initialized from $\mathcal{U}(-\sqrt{k}, \sqrt{k})$ where $k=\frac{1}{\text{in_features}}$.
- `bias` - the learnable bias of shape (`out_features`). If `bias` is `True`, the values are initialized from $\mathcal{U}(-\sqrt{k}, \sqrt{k})$ where $k=\frac{1}{\text{in_features}}$.
___

In [29]:
!python3 -m pytest -v -k "test_nn_linear"

============================= test session starts ==============================
platform linux -- Python 3.7.5, pytest-6.2.5, py-1.10.0, pluggy-1.0.0 -- /home/bowenc/dev/cmu/dlsys/hw2/hw2-env/bin/python3
cachedir: .pytest_cache
rootdir: /home/bowenc/dev/cmu/dlsys/hw2
collected 98 items / 90 deselected / 8 selected                                

tests/test_nn_and_optim.py::test_nn_linear_weight_init_1 PASSED          [ 12%]
tests/test_nn_and_optim.py::test_nn_linear_bias_init_1 PASSED            [ 25%]
tests/test_nn_and_optim.py::test_nn_linear_forward_1 PASSED              [ 37%]
tests/test_nn_and_optim.py::test_nn_linear_forward_2 PASSED              [ 50%]
tests/test_nn_and_optim.py::test_nn_linear_forward_3 PASSED              [ 62%]
tests/test_nn_and_optim.py::test_nn_linear_backward_1 PASSED             [ 75%]
tests/test_nn_and_optim.py::test_nn_linear_backward_2 PASSED             [ 87%]
tests/test_nn_and_optim.py::test_nn_linear_backward_3 PASSED             [100%]

=========

In [41]:
!python3 -m mugrade submit 'eAzORHnc5JIEkr0dqEig' -k "nn_linear"

submit
============================= test session starts ==============================
platform linux -- Python 3.7.5, pytest-6.2.5, py-1.10.0, pluggy-1.0.0
rootdir: /home/bowenc/dev/cmu/dlsys/hw2
collected 22 items / 21 deselected / 1 selected                                

tests/test_nn_and_optim.py 
Submitting nn_linear...
Grader test 1 passed
Grader test 2 passed
Grader test 3 passed
Grader test 4 passed
Grader test 5 passed
Grader test 6 passed
.

======================= 1 passed, 21 deselected in 4.68s =======================


### ReLU
`needle.nn.ReLU(device=None, dtype="float32")`

Applies the rectified linear unit function element-wise:
$ReLU(x) = max(0, x)$.

If you have previously implemented ReLU's backwards pass in terms of itself, note that this is numerically unstable and will likely cause problems
down the line.
Instead, consider that we could write the derivative of ReLU as $I\{x>0\}$, where we arbitrarily decide that the derivative at $x=0$ is 0.
(This is a _subdifferentiable_ function.)

___

In [28]:
!python3 -m pytest -v -k "test_nn_relu"

============================= test session starts ==============================
platform linux -- Python 3.7.5, pytest-6.2.5, py-1.10.0, pluggy-1.0.0 -- /home/bowenc/dev/cmu/dlsys/hw2/hw2-env/bin/python3
cachedir: .pytest_cache
rootdir: /home/bowenc/dev/cmu/dlsys/hw2
collected 98 items / 96 deselected / 2 selected                                

tests/test_nn_and_optim.py::test_nn_relu_forward_1 PASSED                [ 50%]
tests/test_nn_and_optim.py::test_nn_relu_backward_1 PASSED               [100%]

=============================== warnings summary ===============================
hw2-env/lib/python3.7/site-packages/mugrade/mugrade.py:71
  /home/bowenc/dev/cmu/dlsys/hw2/hw2-env/lib/python3.7/site-packages/mugrade/mugrade.py:71: PytestUnknownMarkWarning: Unknown pytest.mark.hookwrapper - is this a typo?  You can register custom marks to avoid this warning - for details, see https://docs.pytest.org/en/stable/mark.html
    @pytest.mark.hookwrapper

-- Docs: https://docs.pytest.org/en/

In [33]:
!python3 -m mugrade submit 'eAzORHnc5JIEkr0dqEig' -k "nn_relu"

submit
============================= test session starts ==============================
platform linux -- Python 3.7.5, pytest-6.2.5, py-1.10.0, pluggy-1.0.0
rootdir: /home/bowenc/dev/cmu/dlsys/hw2
collected 22 items / 21 deselected / 1 selected                                

tests/test_nn_and_optim.py 
Submitting nn_relu...
Grader test 1 passed
Grader test 2 passed
.

======================= 1 passed, 21 deselected in 1.67s =======================


### Sequential
`needle.nn.Sequential(*modules, device=None, dtype="float32")`

Applies a sequence of modules to the input (in the order that they were passed to the constructor) and returns the output of the last module.
These should be kept in a `.module` property: you should _not_ redefine any magic methods like `__getitem__`, as this may not be compatible with our tests.

##### Parameters
- `*modules` - any number of modules of type `needle.nn.Module`

___

In [42]:
!python3 -m pytest -v -k "test_nn_sequential"

============================= test session starts ==============================
platform linux -- Python 3.7.5, pytest-6.2.5, py-1.10.0, pluggy-1.0.0 -- /home/bowenc/dev/cmu/dlsys/hw2/hw2-env/bin/python3
cachedir: .pytest_cache
rootdir: /home/bowenc/dev/cmu/dlsys/hw2
collected 98 items / 96 deselected / 2 selected                                

tests/test_nn_and_optim.py::test_nn_sequential_forward_1 PASSED          [ 50%]
tests/test_nn_and_optim.py::test_nn_sequential_backward_1 PASSED         [100%]

=============================== warnings summary ===============================
hw2-env/lib/python3.7/site-packages/mugrade/mugrade.py:71
  /home/bowenc/dev/cmu/dlsys/hw2/hw2-env/lib/python3.7/site-packages/mugrade/mugrade.py:71: PytestUnknownMarkWarning: Unknown pytest.mark.hookwrapper - is this a typo?  You can register custom marks to avoid this warning - for details, see https://docs.pytest.org/en/stable/mark.html
    @pytest.mark.hookwrapper

-- Docs: https://docs.pytest.org/en/

In [43]:
!python3 -m mugrade submit 'eAzORHnc5JIEkr0dqEig' -k "nn_sequential"

submit
============================= test session starts ==============================
platform linux -- Python 3.7.5, pytest-6.2.5, py-1.10.0, pluggy-1.0.0
rootdir: /home/bowenc/dev/cmu/dlsys/hw2
collected 22 items / 21 deselected / 1 selected                                

tests/test_nn_and_optim.py 
Submitting nn_sequential...
Grader test 1 passed
Grader test 2 passed
.

======================= 1 passed, 21 deselected in 1.73s =======================


### SoftmaxLoss
`needle.nn.SoftmaxLoss(device=None, dtype="float32")`

Applies the softmax loss as defined below (and as implemented in Homework 1), taking in as input a Tensor of logits and a Tensor of the true labels (expressed as a list of numbers, *not* one-hot encoded).

Note that you can use the new `ops.one_hot` function now instead of writing this yourself.
**Importantly**, you should implement your SoftmaxLoss in terms of the `LogSoftmaxOp` you implemented in Q0.
Also note that the equation below is equal to negative log softmax.

\begin{equation}
\ell_\text{softmax}(z,y) = \log \sum_{i=1}^k \exp z_i - z_y
\end{equation}

___

In [66]:
!python3 -m pytest -v -k "test_nn_softmax_loss"

============================= test session starts ==============================
platform linux -- Python 3.7.5, pytest-6.2.5, py-1.10.0, pluggy-1.0.0 -- /home/bowenc/dev/cmu/dlsys/hw2/hw2-env/bin/python3
cachedir: .pytest_cache
rootdir: /home/bowenc/dev/cmu/dlsys/hw2
collected 98 items / 94 deselected / 4 selected                                

tests/test_nn_and_optim.py::test_nn_softmax_loss_forward_1 PASSED        [ 25%]
tests/test_nn_and_optim.py::test_nn_softmax_loss_forward_2 PASSED        [ 50%]
tests/test_nn_and_optim.py::test_nn_softmax_loss_backward_1 PASSED       [ 75%]
tests/test_nn_and_optim.py::test_nn_softmax_loss_backward_2 PASSED       [100%]

=============================== warnings summary ===============================
hw2-env/lib/python3.7/site-packages/mugrade/mugrade.py:71
  /home/bowenc/dev/cmu/dlsys/hw2/hw2-env/lib/python3.7/site-packages/mugrade/mugrade.py:71: PytestUnknownMarkWarning: Unknown pytest.mark.hookwrapper - is this a typo?  You can register cust

In [67]:
!python3 -m mugrade submit 'eAzORHnc5JIEkr0dqEig' -k "nn_softmax_loss"

submit
============================= test session starts ==============================
platform linux -- Python 3.7.5, pytest-6.2.5, py-1.10.0, pluggy-1.0.0
rootdir: /home/bowenc/dev/cmu/dlsys/hw2
collected 22 items / 21 deselected / 1 selected                                

tests/test_nn_and_optim.py 
Submitting nn_softmax_loss...
Grader test 1 passed
Grader test 2 passed
Grader test 3 passed
Grader test 4 passed
.

======================= 1 passed, 21 deselected in 3.61s =======================



### LayerNorm
`needle.nn.LayerNorm(dims, eps=1e-5, device=None, dtype="float32")`

Applies layer normalization over a mini-batch of inputs as described in the paper [Layer Normalization](https://arxiv.org/abs/1607.06450).

\begin{equation}
\hat{z}_{i+1} = \sigma_i(W_i^Tz_i + b_i) \\
z_{i+1} = \frac{\hat{z}_{i+1} - \text{E}[\hat{z}_{i+1}]}{(\text{Var}[\hat{z}_{i+1}]+\epsilon)^{1/2})}
\end{equation}

Note that the affine scaling parameters `weight` and `bias` here are shaped differently than for BatchNorm below.

##### Parameters
- `dims` - input shape
- `eps` - a value added to the denominator for numerical stability.

##### Variables
- `weight` - the learnable weights of shape `dims`, elements initialized to 1.
- `bias` - the learnable bias of shape `dims`, elements initialized to 0.
___

In [ ]:
!python3 -m pytest -v -k "test_nn_layernorm"

In [35]:
!python3 -m mugrade submit 'eAzORHnc5JIEkr0dqEig' -k "nn_layernorm"

submit
============================= test session starts ==============================
platform linux -- Python 3.7.5, pytest-6.2.5, py-1.10.0, pluggy-1.0.0
rootdir: /home/bowenc/dev/cmu/dlsys/hw2
collected 22 items / 21 deselected / 1 selected                                

tests/test_nn_and_optim.py 
Submitting nn_layernorm...
Grader test 1 passed
Grader test 2 passed
Grader test 3 passed
Grader test 4 passed
Grader test 5 passed
Grader test 6 passed
Grader test 7 passed
Grader test 8 passed
.

======================= 1 passed, 21 deselected in 5.31s =======================


### BatchNorm
`needle.nn.BatchNorm(dim, eps=1e-5, momentum=0.1, device=None, dtype="float32")`

Applies batch normalization over a mini-batch of inputs as described in the paper [Batch Normalization: Accelerating Deep Network Training by Reducing Internal Covariate Shift](https://arxiv.org/abs/1502.03167). Unlike layer normalization, normalizes activations over the mini-batch rather than normalizing the activations at each layer. Computes a running average of mean/variance for all features at each layer $\hat{\mu}_{i+1}, \hat{\sigma}^2_{i+1}$, and at test time normalizes by these quantities:

\begin{equation}
(z_{i+1})_j = \frac{(\hat{z}_{i+1})_j - (\hat{\mu}_{i+1})_j}{((\hat{\sigma}^2_{i+1})_j+\epsilon)^{1/2}}
\end{equation}

You will additionally have to write the function `_child_modules` in `nn.py` in order to make
it possible to set a flag on a module and its children which indicates whether or not training is in progress;
that is, all modules have a property `training` which should be false after calling `model.eval()` on the module or one of its parents, and vice versa for `model.train()`. (There is a test for this in this section.)

BatchNorm uses the running estimates of mean and variance instead of batch statistics at test time, i.e.,
after `model.eval()` has been called on the BatchNorm layer's `training` flag is false.

**Important:** A small detail here is that our implementation of BatchNorm uses the *biased* estimate
of the variance during training, but computes a running estimate of the *unbiased* version. The biased estimate
divides by $N$, while the unbiased estimate divides by $N-1$.

To compute the running estimates, you can use the equation $$\hat{x_{new}} = (1 - m) \hat{x_{old}} + mx_{observed},$$
where $m$ is momentum.

##### Parameters
- `dim` - input dim
- `eps` - a value added to the denominator for numerical stability.
- `momentum` - the value used for the running mean and running variance computation.

##### Variables
- `weight` - the learnable weights of size `dim`, elements initialized to 1.
- `bias` - the learnable bias of size `dim`, elements initialized to 0.
- `running_mean` - the running mean used at evaluation time, elements initialized to 0.
- `running_var` - the running (unbiased) variance used at evaluation time, elements initialized to 1. 

___

In [24]:
!python3 -m pytest -v -k "test_nn_batchnorm"

============================= test session starts ==============================
platform linux -- Python 3.7.5, pytest-6.2.5, py-1.10.0, pluggy-1.0.0 -- /home/bowenc/dev/cmu/dlsys/hw2/hw2-env/bin/python3
cachedir: .pytest_cache
rootdir: /home/bowenc/dev/cmu/dlsys/hw2
collected 98 items / 77 deselected / 21 selected                               

tests/test_nn_and_optim.py::test_nn_batchnorm_check_model_eval_switches_training_flag_1 PASSED [  4%]
tests/test_nn_and_optim.py::test_nn_batchnorm_forward_1 PASSED           [  9%]
tests/test_nn_and_optim.py::test_nn_batchnorm_forward_2 PASSED           [ 14%]
tests/test_nn_and_optim.py::test_nn_batchnorm_forward_3 PASSED           [ 19%]
tests/test_nn_and_optim.py::test_nn_batchnorm_forward_4 PASSED           [ 23%]
tests/test_nn_and_optim.py::test_nn_batchnorm_forward_affine_1 PASSED    [ 28%]
tests/test_nn_and_optim.py::test_nn_batchnorm_forward_affine_2 PASSED    [ 33%]
tests/test_nn_and_optim.py::test_nn_batchnorm_forward_affine_3 PASSE

In [36]:
!python3 -m mugrade submit 'eAzORHnc5JIEkr0dqEig' -k "nn_batchnorm"

submit
============================= test session starts ==============================
platform linux -- Python 3.7.5, pytest-6.2.5, py-1.10.0, pluggy-1.0.0
rootdir: /home/bowenc/dev/cmu/dlsys/hw2
collected 22 items / 21 deselected / 1 selected                                

tests/test_nn_and_optim.py 
Submitting nn_batchnorm...
Grader test 1 passed
Grader test 2 passed
Grader test 3 passed
Grader test 4 passed
Grader test 5 passed
Grader test 6 passed
Grader test 7 passed
Grader test 8 passed
[needle.Tensor(14.9997835), needle.Tensor([[0.2546954  0.01621025 0.6249844 ]
 [0.85605955 0.87175167 0.89997756]
 [1.7829783  0.65021145 2.0249493 ]
 [0.95610255 1.7308954  0.2249943 ]
 [1.1501089  1.730895   1.2249695 ]]), needle.Tensor([[ 0.50467354  0.12731948  0.79055953]
 [ 0.92523485 -0.9336764  -0.94867146]
 [-1.3352821   0.8063569  -1.4230071 ]
 [ 0.97780496 -1.315635    0.47433564]
 [-1.0724313   1.3156348   1.1067834 ]]), needle.Tensor([[0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]

Grader test 10 passed
[needle.Tensor(11.999924), needle.Tensor([[[[6.2027740e+00 2.0752174e-01 5.4983473e-01]
   [4.9633168e-02 1.3008002e+00 3.3091348e-01]]]


 [[[3.9914376e-03 9.1559835e-02 9.1559835e-02]
   [1.2856747e+00 1.3934101e+00 4.9225160e-01]]]]), needle.Tensor([[[[-2.490537   -0.45554554  0.7415084 ]
   [ 0.22278503  1.1405263  -0.5752508 ]]]


 [[[ 0.06317782  0.30258855  0.30258855]
   [-1.133876    1.1804279   0.70160645]]]]), needle.Tensor([[[[0. 0. 0.]
   [0. 0. 0.]]]


 [[[0. 0. 0.]
   [0. 0. 0.]]]]), needle.Tensor([[[[0.]]]]), needle.Tensor([0.]), needle.Tensor([[[[-2.490537   -0.45554554  0.7415084 ]
   [ 0.22278503  1.1405263  -0.5752508 ]]]


 [[[ 0.06317782  0.30258855  0.30258855]
   [-1.133876    1.1804279   0.70160645]]]]), needle.Tensor([[[[1.2530766 1.2530766 1.2530766]
   [1.2530766 1.2530766 1.2530766]]]


 [[[1.2530766 1.2530766 1.2530766]
   [1.2530766 1.2530766 1.2530766]]]]), needle.Tensor([[[[1.2530766]]]]), needle.Tensor([[[[1.5702009]]]]), needle.T

Grader test 12 passed
Grader test 13 passed
Grader test 14 passed
Grader test 15 passed
Grader test 16 passed
{'training': False, 'dim': 3, 'eps': 1e-05, 'momentum': 0.1, 'weight': needle.Tensor([1. 1. 1.]), 'bias': needle.Tensor([0. 0. 0.]), 'running_mean': needle.Tensor([1.5894531 1.7175173 1.6310714]), 'running_var': needle.Tensor([1.7184839 1.5241226 1.9188087])}
[needle.Tensor(23.999916), needle.Tensor([[[0.5037282  1.1602125 ]
  [0.5680535  0.12473619]
  [0.84278774 1.3760325 ]]

 [[3.4671102  0.16739807]
  [0.97361475 0.8620431 ]
  [1.6914275  0.7081759 ]]

 [[0.195844   2.2339013 ]
  [0.671141   1.0319458 ]
  [0.04161911 1.0404787 ]]

 [[0.195844   0.07592534]
  [3.2746246  0.49381396]
  [1.258979   1.0404787 ]]]), needle.Tensor([[[ 0.70973814  1.0771316 ]
  [ 0.7536932  -0.3531801 ]
  [-0.91803473  1.1730441 ]]

 [[-1.8620178   0.4091431 ]
  [ 0.9867192   0.92846274]
  [ 1.3005489  -0.8415319 ]]

 [[ 0.44254264 -1.4946241 ]
  [-0.819232    1.0158473 ]
  [ 0.20400763 -1.0200386

### Dropout
`needle.nn.Dropout(drop_prob, device=None, dtype="float32")`

During training, randomly zeroes some of the elements of the input tensor with probability `drop_prob` using samples from a Bernoulli distribution. This has proven to be an effective technique for regularization and preventing the co-adaptation of neurons as described in the paper [Improving neural networks by preventing co-adaption of feature detectors](https://arxiv.org/abs/1207.0580). During evaluation the module simply computes an identity function. 

\begin{equation}
\hat{z}_{i+1} = \sigma_i (W_i^T z_i + b_i) \\
(z_{i+1})_j = 
    \begin{cases}
    (\hat{z}_{i+1})_j /(1-p) & \text{with probability } 1-p \\
    0 & \text{with probability } p \\
    \end{cases}
\end{equation}

**Important**: If the Dropout module the flag `training=False`, you shouldn't "dropout" any weights. That is, dropout applies during training only, not during evaluation.

##### Parameters
- `drop_prob` - probability of an element to be zeroed. 

___

In [9]:
!python3 -m pytest -v -k "test_nn_dropout"

============================= test session starts ==============================
platform linux -- Python 3.7.5, pytest-6.2.5, py-1.10.0, pluggy-1.0.0 -- /home/bowenc/dev/cmu/dlsys/hw2/hw2-env/bin/python3
cachedir: .pytest_cache
rootdir: /home/bowenc/dev/cmu/dlsys/hw2
collected 98 items / 96 deselected / 2 selected                                

tests/test_nn_and_optim.py::test_nn_dropout_forward_1 PASSED             [ 50%]
tests/test_nn_and_optim.py::test_nn_dropout_backward_1 PASSED            [100%]

=============================== warnings summary ===============================
hw2-env/lib/python3.7/site-packages/mugrade/mugrade.py:71
  /home/bowenc/dev/cmu/dlsys/hw2/hw2-env/lib/python3.7/site-packages/mugrade/mugrade.py:71: PytestUnknownMarkWarning: Unknown pytest.mark.hookwrapper - is this a typo?  You can register custom marks to avoid this warning - for details, see https://docs.pytest.org/en/stable/mark.html
    @pytest.mark.hookwrapper

-- Docs: https://docs.pytest.org/en/

In [32]:
!python3 -m mugrade submit 'eAzORHnc5JIEkr0dqEig' -k "nn_dropout"

submit
============================= test session starts ==============================
platform linux -- Python 3.7.5, pytest-6.2.5, py-1.10.0, pluggy-1.0.0
rootdir: /home/bowenc/dev/cmu/dlsys/hw2
collected 22 items / 21 deselected / 1 selected                                

tests/test_nn_and_optim.py 
Submitting nn_dropout...
Grader test 1 passed
Grader test 2 passed
.

======================= 1 passed, 21 deselected in 2.03s =======================


### Residual
`needle.nn.Residual(fn: Module, device=None, dtype="float32")`

Applies a residual or skip connection given module $\mathcal{F}$ and input Tensor $x$, returning $\mathcal{F}(x) + x$.
##### Parameters
- `fn` - module of type `needle.nn.Module`

In [43]:
!python3 -m pytest -v -k "test_nn_residual"

============================= test session starts ==============================
platform linux -- Python 3.7.5, pytest-6.2.5, py-1.10.0, pluggy-1.0.0 -- /home/bowenc/dev/cmu/dlsys/hw2/hw2-env/bin/python3
cachedir: .pytest_cache
rootdir: /home/bowenc/dev/cmu/dlsys/hw2
collected 98 items / 96 deselected / 2 selected                                

tests/test_nn_and_optim.py::test_nn_residual_forward_1 PASSED            [ 50%]
tests/test_nn_and_optim.py::test_nn_residual_backward_1 PASSED           [100%]

=============================== warnings summary ===============================
hw2-env/lib/python3.7/site-packages/mugrade/mugrade.py:71
  /home/bowenc/dev/cmu/dlsys/hw2/hw2-env/lib/python3.7/site-packages/mugrade/mugrade.py:71: PytestUnknownMarkWarning: Unknown pytest.mark.hookwrapper - is this a typo?  You can register custom marks to avoid this warning - for details, see https://docs.pytest.org/en/stable/mark.html
    @pytest.mark.hookwrapper

-- Docs: https://docs.pytest.org/en/

In [44]:
!python3 -m mugrade submit 'eAzORHnc5JIEkr0dqEig' -k "nn_residual"

submit
============================= test session starts ==============================
platform linux -- Python 3.7.5, pytest-6.2.5, py-1.10.0, pluggy-1.0.0
rootdir: /home/bowenc/dev/cmu/dlsys/hw2
collected 22 items / 21 deselected / 1 selected                                

tests/test_nn_and_optim.py 
Submitting nn_residual...
Grader test 1 passed
[needle.Tensor(22.030226), needle.Tensor([[ 1.9047918  -0.64011514  2.8450792 ]
 [ 4.3380623   0.31210983  0.32276547]
 [ 0.5470216   3.0703483   0.32774067]
 [ 5.52239     3.0602338   0.4197983 ]]), needle.Tensor([[ 0.20479175 -0.7901151  -0.3549208 ]
 [-0.01193756 -0.5378902  -0.52723455]
 [ 0.49702162 -0.87965184  0.12774065]
 [ 0.57239026 -1.0397661  -0.13020168]]), needle.Tensor([ 0.09241457 -0.4535496   0.10754485]), needle.Tensor([[ 0.11237718 -0.33656555 -0.46246564]
 [-0.10435212 -0.08434059 -0.6347794 ]
 [ 0.40460706 -0.42610225  0.02019579]
 [ 0.47997567 -0.5862165  -0.23774654]]), needle.Tensor([[-0.19575776  0.02475643 -0.068

## Question 3 [30 points]

Implement the `step` function of the following optimizers.
Make sure that your optimizers _don't_ modify the gradients of tensors in-place.

We have included some tests to ensure that you are not consuming excessive memory, which can happen if you are
not using `.data` or `.detach()` in the right places, thus building an increasingly large computational graph
(not just in the optimizers, but in the previous modules as well).
You can ignore these tests, which include the string `check_memory` at your own discretion.

___

### SGD
`needle.optim.SGD(params, lr=0.01, momentum=0.9, weight_decay=0.0)`

Implements stochastic gradient descent (optionally with momentum, shown as $\beta$ below). 

\begin{equation}
\begin{split}
    u_{t+1} &= \beta u_t + \nabla_\theta f(\theta_t) \\
    \theta_{t+1} &= \theta_t - \alpha u_{t+1}
\end{split}
\end{equation}

##### Parameters
- `params` - iterable of parameters of type `needle.nn.Parameter` to optimize
- `lr` (*float*) - learning rate
- `momentum` (*float*) - momentum factor
- `weight_decay` (*float*) - weight decay (L2 penalty)
___

In [33]:
!python3 -m pytest -v -k "test_optim_sgd"

============================= test session starts ==============================
platform linux -- Python 3.7.5, pytest-6.2.5, py-1.10.0, pluggy-1.0.0 -- /home/bowenc/dev/cmu/dlsys/hw2/hw2-env/bin/python3
cachedir: .pytest_cache
rootdir: /home/bowenc/dev/cmu/dlsys/hw2
collected 98 items / 92 deselected / 6 selected                                

tests/test_nn_and_optim.py::test_optim_sgd_vanilla_1 PASSED              [ 16%]
tests/test_nn_and_optim.py::test_optim_sgd_momentum_1 PASSED             [ 33%]
tests/test_nn_and_optim.py::test_optim_sgd_weight_decay_1 PASSED         [ 50%]
tests/test_nn_and_optim.py::test_optim_sgd_momentum_weight_decay_1 PASSED [ 66%]
tests/test_nn_and_optim.py::test_optim_sgd_layernorm_residual_1 PASSED   [ 83%]
tests/test_nn_and_optim.py::test_optim_sgd_z_memory_check_1 PASSED       [100%]

=============================== warnings summary ===============================
hw2-env/lib/python3.7/site-packages/mugrade/mugrade.py:71
  /home/bowenc/dev/cmu/dlsys/

In [74]:
!python3 -m mugrade submit 'eAzORHnc5JIEkr0dqEig' -k "optim_sgd"

submit
============================= test session starts ==============================
platform linux -- Python 3.7.5, pytest-6.2.5, py-1.10.0, pluggy-1.0.0
rootdir: /home/bowenc/dev/cmu/dlsys/hw2
collected 22 items / 21 deselected / 1 selected                                

tests/test_nn_and_optim.py 
Submitting optim_sgd...
Grader test 1 passed
Grader test 2 passed
F

=================================== FAILURES ===================================
_______________________________ submit_optim_sgd _______________________________

    def submit_optim_sgd():
    	mugrade.submit(learn_model_1d(48, 17, lambda z: nn.Sequential(nn.Linear(48, 32), nn.ReLU(), nn.Linear(32, 17)), ndl.optim.SGD, lr=0.03, momentum=0.0, epochs=2))
    	mugrade.submit(learn_model_1d(48, 16, lambda z: nn.Sequential(nn.Linear(48, 32), nn.ReLU(), nn.Linear(32, 16)), ndl.optim.SGD, lr=0.01, momentum=0.9, epochs=2))
>   	mugrade.submit(learn_model_1d(48, 16, lambda z: nn.Sequential(nn.Linear(48, 32), nn.ReLU(), nn.B

### Adam
`needle.optim.Adam(params, lr=0.01, beta1=0.9, beta2=0.999, eps=1e-8, weight_decay=0.0)`

Implements Adam algorithm, proposed in [Adam: A Method for Stochastic Optimization](https://arxiv.org/abs/1412.6980). 

\begin{equation}
\begin{split}
u_{t+1} &= \beta_1 u_t + (1-\beta_1) \nabla_\theta f(\theta_t) \\
v_{t+1} &= \beta_2 v_t + (1-\beta_2) (\nabla_\theta f(\theta_t))^2 \\
\hat{u_{t+1}} &= u_{t+1} / (1 - \beta_1^t) \quad \text{(bias correction)} \\
\hat{v_{t+1}} &= v_{t+1} / (1 - \beta_2^t) \quad \text{(bias correction)}\\
\theta_{t+1} &= \theta_t - \alpha \hat{u_{t+1}}/(\hat{v_{t+1}}^{1/2}+\epsilon)
\end{split}
\end{equation}

**Important:** Pay attention to whether or not you are applying bias correction.

##### Parameters
- `params` - iterable of parameters of type `needle.nn.Parameter` to optimize
- `lr` (*float*) - learning rate
- `beta1` (*float*) - coefficient used for computing running average of gradient
- `beta2` (*float*) - coefficient used for computing running average of square of gradient
- `eps` (*float*) - term added to the denominator to improve numerical stability
- `bias_correction` - whether to use bias correction for $u, v$
- `weight_decay` (*float*) - weight decay (L2 penalty)

In [23]:
!python3 -m pytest -v -k "test_optim_adam"

============================= test session starts ==============================
platform linux -- Python 3.7.5, pytest-6.2.5, py-1.10.0, pluggy-1.0.0 -- /home/bowenc/dev/cmu/dlsys/hw2/hw2-env/bin/python3
cachedir: .pytest_cache
rootdir: /home/bowenc/dev/cmu/dlsys/hw2
collected 98 items / 91 deselected / 7 selected                                

tests/test_nn_and_optim.py::test_optim_adam_1 PASSED                     [ 14%]
tests/test_nn_and_optim.py::test_optim_adam_weight_decay_1 PASSED        [ 28%]
tests/test_nn_and_optim.py::test_optim_adam_batchnorm_1 PASSED           [ 42%]
tests/test_nn_and_optim.py::test_optim_adam_batchnorm_eval_mode_1 PASSED [ 57%]
tests/test_nn_and_optim.py::test_optim_adam_layernorm_1 PASSED           [ 71%]
tests/test_nn_and_optim.py::test_optim_adam_weight_decay_bias_correction_1 PASSED [ 85%]
tests/test_nn_and_optim.py::test_optim_adam_z_memory_check_1 PASSED      [100%]

=============================== warnings summary ===============================

In [24]:
!python3 -m mugrade submit 'eAzORHnc5JIEkr0dqEig' -k "optim_adam"

submit
============================= test session starts ==============================
platform linux -- Python 3.7.5, pytest-6.2.5, py-1.10.0, pluggy-1.0.0
rootdir: /home/bowenc/dev/cmu/dlsys/hw2
collected 22 items / 21 deselected / 1 selected                                

tests/test_nn_and_optim.py 
Submitting optim_adam...
Grader test 1 passed
Grader test 2 passed
Grader test 3 passed
Grader test 4 passed
Grader test 5 passed
Grader test 6 passed
.

======================= 1 passed, 21 deselected in 5.81s =======================


## Question 4 [10 points]

In this question, you will implement two data primitives: `needle.data.DataLoader` and `needle.data.Dataset`. `Dataset` stores the samples and their corresponding labels, and `DataLoader` wraps an iterable around the `Dataset` to enable easy access to the samples. 

For this question, you will be working in `python/needle/data.py`. First, copy your solution to `parse_mnist` from the previous homework into the `parse_mnist` function. 

### Transformations

First we will implement a few transformations that are helpful when working with images. We will stick with a horizontal flip and a random crop for now. Fill out the following functions in `data.py`.
___ 

#### FlipHorizontal
`needle.data.FlipHorizontal()`

Flips the image horizontally.
___

#### RandomCrop
`needle.data.RandomCrop(padding=3)`

Padding is added to all side of the image, and then the image is cropped back to it's original size at a random location. Returns an image the same size as the original image.

##### Parameters
- `padding` (*int*) - The padding on each border of the image.

In [9]:
!python3 -m pytest -v -k "flip_horizontal"
!python3 -m pytest -v -k "random_crop"

============================= test session starts ==============================
platform linux -- Python 3.7.5, pytest-6.2.5, py-1.10.0, pluggy-1.0.0 -- /home/bowenc/dev/cmu/dlsys/hw2/hw2-env/bin/python3
cachedir: .pytest_cache
rootdir: /home/bowenc/dev/cmu/dlsys/hw2
collected 98 items / 97 deselected / 1 selected                                

tests/test_data.py::test_flip_horizontal PASSED                          [100%]

=============================== warnings summary ===============================
hw2-env/lib/python3.7/site-packages/mugrade/mugrade.py:71
  /home/bowenc/dev/cmu/dlsys/hw2/hw2-env/lib/python3.7/site-packages/mugrade/mugrade.py:71: PytestUnknownMarkWarning: Unknown pytest.mark.hookwrapper - is this a typo?  You can register custom marks to avoid this warning - for details, see https://docs.pytest.org/en/stable/mark.html
    @pytest.mark.hookwrapper

-- Docs: https://docs.pytest.org/en/stable/warnings.html
================= 1 passed, 97 deselected, 1 warning in 0.1

In [10]:
!python3 -m mugrade submit 'eAzORHnc5JIEkr0dqEig' -k "flip_horizontal"
!python3 -m mugrade submit 'eAzORHnc5JIEkr0dqEig' -k "random_crop"

submit
============================= test session starts ==============================
platform linux -- Python 3.7.5, pytest-6.2.5, py-1.10.0, pluggy-1.0.0
rootdir: /home/bowenc/dev/cmu/dlsys/hw2
collected 22 items / 21 deselected / 1 selected                                

tests/test_data.py 
Submitting flip_horizontal...
Grader test 1 passed
Grader test 2 passed
Grader test 3 passed
Grader test 4 passed
Grader test 5 passed
Grader test 6 passed
Grader test 7 passed
Grader test 8 passed
Grader test 9 passed
Grader test 10 passed
.

======================= 1 passed, 21 deselected in 7.08s =======================
submit
============================= test session starts ==============================
platform linux -- Python 3.7.5, pytest-6.2.5, py-1.10.0, pluggy-1.0.0
rootdir: /home/bowenc/dev/cmu/dlsys/hw2
collected 22 items / 21 deselected / 1 selected                                

tests/test_data.py 
Submitting random_crop...
1 1
Grader test 1 passed
1 1
Grader test 2 passed
1

### Dataset

Each `Dataset` subclass must implement three functions: `__init__`, `__len__`, and `__getitem__`. The `__init__` function initializes the images, labels, and transforms. The `__len__` function returns the number of samples in the dataset. The `__getitem__` function retrieves a sample from the dataset at a given index `idx`, calls the transform functions on the image (if applicable), converts the image and label to a numpy array (the data will be converted to Tensors elsewhere). Fill out these functions in the `MNISTDataset` class: 
___ 

### MNISTDataset
`needle.data.MNISTDataset(image_filesname, label_filesname)`

##### Parameters
- `image_filesname` - path of file containing images
- `label_filesname` - path of file containing labels


In [14]:
!python3 -m pytest -v -k "mnist_dataset"

============================= test session starts ==============================
platform linux -- Python 3.7.5, pytest-6.2.5, py-1.10.0, pluggy-1.0.0 -- /home/bowenc/dev/cmu/dlsys/hw2/hw2-env/bin/python3
cachedir: .pytest_cache
rootdir: /home/bowenc/dev/cmu/dlsys/hw2
collected 98 items / 97 deselected / 1 selected                                

tests/test_data.py::test_mnist_dataset PASSED                            [100%]

=============================== warnings summary ===============================
hw2-env/lib/python3.7/site-packages/mugrade/mugrade.py:71
  /home/bowenc/dev/cmu/dlsys/hw2/hw2-env/lib/python3.7/site-packages/mugrade/mugrade.py:71: PytestUnknownMarkWarning: Unknown pytest.mark.hookwrapper - is this a typo?  You can register custom marks to avoid this warning - for details, see https://docs.pytest.org/en/stable/mark.html
    @pytest.mark.hookwrapper

-- Docs: https://docs.pytest.org/en/stable/warnings.html
=========== 1 passed, 97 deselected, 1 warning in 117.59s (

In [4]:
!python3 -m mugrade submit 'eAzORHnc5JIEkr0dqEig' -k "mnist_dataset"

submit
============================= test session starts ==============================
platform linux -- Python 3.7.5, pytest-6.2.5, py-1.10.0, pluggy-1.0.0
rootdir: /home/bowenc/dev/cmu/dlsys/hw2
collected 22 items / 21 deselected / 1 selected                                

tests/test_data.py 
Submitting mnist_dataset...
Grader test 1 passed
Grader test 2 passed
Grader test 3 passed
Grader test 4 passed
Grader test 5 passed
Grader test 6 passed
.

====================== 1 passed, 21 deselected in 37.19s =======================


### Sampling 

During training, we typically want to pass samples in mini-batches, and shuffle the data at each epoch to reduce model overfitting. The dataloader we will eventually build will be flexible enough to change how it samples points from a given dataset, as well as how it combines the data at a batch level. First, we define a `Sampler` class that will be subclassed to build different data sampling strategies. In this homework we will extend this and create a `SequentialSampler` and a `RandomSampler`. Furthermore, we typically batch our data. Therefore, we will also construct a `BatchSampler`, which will take a `Sampler` and return the sampled indexes as batches instead of individual indexes. Each iteration of a sampler will return an index, whereas each iteration of a batch sampler will return a list of indexes. Fill out the following classes in `python/needle/data.py`.
___

### SequentialSampler
`needle.data.SequentialSampler(data_source: needle.data.Dataset)`

Samples elements sequentially, always in the same order. 

##### Parameters
- `data_source` - `needle.data.Dataset` - dataset 
___ 

### RandomSampler
`needle.data.RandomSampler(data_source: Sized, replacement: bool = False,
                 num_samples: Optional[int] = None)`

Samples elements randomly. If replacement is specified then you must also specify `num_samples`. If replacement is not specified, then the `data_source` size will be used as `num_samples` and all indices shuffled. 

##### Parameters
- `data_source` - `needle.data.Dataset` - dataset 
- `replacement` - `bool` - whether or not to use replacement when sampling from the dataset
- `num_samples` - `int` - if using replacement, how many samples to draw
___ 

### BatchSampler
`needle.data.BatchSampler(sampler: Union[Sampler, Iterable], batch_size: int, drop_last: bool)`

Given a sampler, batch the sampled indexes in sets of `batch_size`. If `drop_last` is set to `True`, and the final batch is not full, then it is dropped. 

##### Parameters
- `sampler` - `needle.data.Sampler` - the sampler who's output is to be batched
- `batch_size` - `int` - the batch size, or the set size to group the indexes in
- `drop_last` - `bool` - if set, the final batch is dropped if it is less than `batch_size`
___ 


In [15]:
!python3 -m pytest -v -k "sequential_sampler"
!python3 -m pytest -v -k "random_sampler"
!python3 -m pytest -v -k "batch_sampler"

============================= test session starts ==============================
platform linux -- Python 3.7.5, pytest-6.2.5, py-1.10.0, pluggy-1.0.0 -- /home/bowenc/dev/cmu/dlsys/hw2/hw2-env/bin/python3
cachedir: .pytest_cache
rootdir: /home/bowenc/dev/cmu/dlsys/hw2
collected 98 items / 97 deselected / 1 selected                                

tests/test_data.py::test_sequential_sampler PASSED                       [100%]

=============================== warnings summary ===============================
hw2-env/lib/python3.7/site-packages/mugrade/mugrade.py:71
  /home/bowenc/dev/cmu/dlsys/hw2/hw2-env/lib/python3.7/site-packages/mugrade/mugrade.py:71: PytestUnknownMarkWarning: Unknown pytest.mark.hookwrapper - is this a typo?  You can register custom marks to avoid this warning - for details, see https://docs.pytest.org/en/stable/mark.html
    @pytest.mark.hookwrapper

-- Docs: https://docs.pytest.org/en/stable/warnings.html
================= 1 passed, 97 deselected, 1 warning in 0.1

In [17]:
!python3 -m mugrade submit 'eAzORHnc5JIEkr0dqEig' -k "sequential_sampler"
!python3 -m mugrade submit 'eAzORHnc5JIEkr0dqEig' -k "random_sampler"
!python3 -m mugrade submit 'eAzORHnc5JIEkr0dqEig' -k "batch_sampler"

submit
============================= test session starts ==============================
platform linux -- Python 3.7.5, pytest-6.2.5, py-1.10.0, pluggy-1.0.0
rootdir: /home/bowenc/dev/cmu/dlsys/hw2
collected 22 items / 21 deselected / 1 selected                                

tests/test_data.py 
Submitting sequential_sampler...
Grader test 1 passed
Grader test 2 passed
.

======================= 1 passed, 21 deselected in 1.86s =======================
submit
============================= test session starts ==============================
platform linux -- Python 3.7.5, pytest-6.2.5, py-1.10.0, pluggy-1.0.0
rootdir: /home/bowenc/dev/cmu/dlsys/hw2
collected 22 items / 21 deselected / 1 selected                                

tests/test_data.py 
Submitting random_sampler...
Grader test 1 passed
.

======================= 1 passed, 21 deselected in 1.11s =======================
submit
============================= test session starts ==============================
platform linux -- Pyt

### Collating

So far we have a `Dataset`, a `Sampler` to draw data points from the dataset, and we can even batch these data points by wrapping our `Sampler` in a `BatchSampler`. However, there is a final step we need to perform before serving data points: collation. Our data is not necessarily in the exact format or structure that we want to leverage during training, and these specifications can differ. Here we will write a collator function for the MNIST dataset. This function takes a batch of data and refactors it such that we return the input data and label separately, the outer dimension of these returned variables is the batch size, and they are of the proper data type (`Tensor` (*float64*)). Fill out the following function in `data.py`.
___
#### collate_mnist
`needle.data.collate_mnist(batch)`

Take a batch of data and transform it into a Tensor of the proper shape.

##### Parameters
- `batch` - a batch of data, can be of different types including a tuple or a list. 

In [18]:
!python3 -m pytest -v -k "collate_mnist"

============================= test session starts ==============================
platform linux -- Python 3.7.5, pytest-6.2.5, py-1.10.0, pluggy-1.0.0 -- /home/bowenc/dev/cmu/dlsys/hw2/hw2-env/bin/python3
cachedir: .pytest_cache
rootdir: /home/bowenc/dev/cmu/dlsys/hw2
collected 98 items / 97 deselected / 1 selected                                

tests/test_data.py::test_collate_mnist PASSED                            [100%]

=============================== warnings summary ===============================
hw2-env/lib/python3.7/site-packages/mugrade/mugrade.py:71
  /home/bowenc/dev/cmu/dlsys/hw2/hw2-env/lib/python3.7/site-packages/mugrade/mugrade.py:71: PytestUnknownMarkWarning: Unknown pytest.mark.hookwrapper - is this a typo?  You can register custom marks to avoid this warning - for details, see https://docs.pytest.org/en/stable/mark.html
    @pytest.mark.hookwrapper

-- Docs: https://docs.pytest.org/en/stable/warnings.html
================= 1 passed, 97 deselected, 1 warning in 16.

In [3]:
!python3 -m mugrade submit 'eAzORHnc5JIEkr0dqEig' -k "collate_mnist"

submit
============================= test session starts ==============================
platform linux -- Python 3.7.5, pytest-6.2.5, py-1.10.0, pluggy-1.0.0
rootdir: /home/bowenc/dev/cmu/dlsys/hw2
collected 22 items / 21 deselected / 1 selected                                

tests/test_data.py 
Submitting collate_mnist...
Grader test 1 passed
Grader test 2 passed
Grader test 3 passed
Grader test 4 passed
Grader test 5 passed
Grader test 6 passed
.

======================= 1 passed, 21 deselected in 4.56s =======================


### Dataloader

We now have all of the components we need to create a dataloader. Using all of the prior objects and methods created in this question, we will build a dataloader object to combine them together. We have provided a base `Dataloader` implementation that is flexible enough to include multiprocess loading of samples (without multithreading, data loading can become the bottle neck when using fast computation methods). For simplicity we will only implement a single threaded dataloader, but we hope that for those interested in how to extend this to a multi-threaded process, the "jump-off" points are clear. This also applies to the underlying data type. In this homework we focus on `Iterable` datasets, but commonly `Map` style datasets are used as well. Again, we hope that the base implemntation makes it clear how one would extend this to work with `Map` style datasets, although we only work with `Iterable` ones here. Look through the `DataLoader` class, as well as the classes it calls/depends on. Then fill out the `fetch` method of `_IterableDatasetFetcher`. 
___

### Dataloader
`needle.data.Dataloader(dataset: Dataset, batch_size: Optional[int] = 1, shuffle: bool = False, sampler: Union[Sampler, Iterable, None] = None, collate_fn: Optional = default_collate, drop_last: bool = False,`

Combine a dataset, sampler, and collator to serve datapoints from a dataset. 

##### Parameters
- `dataset` - `needle.data.Dataset` - a dataset 
- `batch_size` - `int` - what batch size to serve the data in 
- `shuffle` - `bool` - if a sampler is not provided, use this to choose between a `SequentialSampler` and a `RandomSampler`
- `collate_fn` - `Generic` - the function to use to collate the samples. For simplicity we will be using only the collate_mnist here, but a general collation function can be built. 
- `drop_last` - `bool` - whether or not to drop the last batch if it is not full. 
___ 

### \_IterableDatasetFetcher
`needle.data._IterableDatasetFetcher(dataset, collate_fn, drop_last)`

This function takes a batch of indexes (typically returned from a `Sampler`), acquires the data from the `dataset`, collates the data using the `collate_fn`, and then returns it for use.  

##### Parameters
- `dataset` - `needle.data.Dataset` - a dataset 
- `collate_fn` - `Generic` - the function to use to collate the samples. For simplicity we will be using only the `collate_mnist` here, but a general collation function can be built. 
- `drop_last` - `bool` - whether or not to drop the last batch if it is not full. 
___ 



In [ ]:
!python3 -m pytest -v -k "test_dataloader"

In [19]:
!python3 -m mugrade submit 'eAzORHnc5JIEkr0dqEig' -k "dataloader"

submit
============================= test session starts ==============================
platform linux -- Python 3.7.5, pytest-6.2.5, py-1.10.0, pluggy-1.0.0
rootdir: /home/bowenc/dev/cmu/dlsys/hw2
collected 22 items / 21 deselected / 1 selected                                

tests/test_data.py 
Submitting dataloader...
Grader test 1 passed
Grader test 2 passed
Grader test 3 passed
Grader test 4 passed
.

====================== 1 passed, 21 deselected in 22.38s =======================


## Question 5 [20 points]

Given you have now implemented all the necessary components for our neural network library, let's build and train an MLP ResNet. For this question, you will be working in `apps/mlp_resnet.py`. First, fill out the functions `ResidualBlock` and `MLPResNet` as described below:

### ResidualBlock
`ResidualBlock(dim, hidden_dim, norm=nn.BatchNorm, drop_prob=0.9)`

Implements a residual block as follows:
![](figures/residualblock.png)
where the first linear layer has `in_features=dim` and `out_features=hidden_dim`, and the last linear layer has `out_features=dim`. Returns the block as type `nn.Module`. 

##### Parameters
- `dim` (*int*) - input dim
- `hidden_dim` (*int*) - hidden dim
- `norm` (*nn.Module*) - normalization method
- `drop_prob` (*float*) - dropout probability

___

### MLPResNet
`ResidualBlock(dim, hidden_dim=100, num_blocks=3, num_classes=10, norm=nn.BatchNorm, drop_prob=0.1)`

Implements an MLP ResNet as follows:
![](figures/mlp_resnet.png)
where the first linear layer has `in_features=dim` and `out_features=hidden_dim`, and each ResidualBlock has `dim=hidden_dim` and `hidden_dim=hidden_dim//2`. Returns a network of type `nn.Module`. __NOTE__: please hard-code `drop_prob` to 0.1 (or ignore `drop_prob`) in this function due to an error in the reference implementation.

##### Parameters
- `dim` (*int*) - input dim
- `hidden_dim` (*int*) - hidden dim
- `num_blocks` (*int*) - number of ResidualBlocks
- `num_classes` (*int*) - number of classes
- `norm` (*nn.Module*) - normalization method
- `drop_prob` (*float*) - dropout probability (0.1)
___ 

Once you have the deep learning model architecture correct, let's train the network using our new neural network library components. Specifically, implement the functions `train_epoch`, `evaluate` and `train_mnist`. 

___
`train_epoch(dataloader, model, loss_fn, opt)`

Executes one epoch of training, iterating over the entire training dataset once (just like `nn_epoch` from previous homeworks). Returns the average accuracy (as a *float*) and the average loss over all samples (as a *float*). Sets the model to `training` mode at the beginning of the function.  

##### Parameters
- `dataloader` (*`needle.data.DataLoader`*) - dataloader returning samples from the training dataset
- `model` (*`needle.nn.Module`*) - neural network
- `loss_fn` (*`needle.nn.Module` type*) - loss function to optimize over
- `opt` (*`needle.optim.Optimizer`*) - optimizer instance

___
`evaluate(dataloader, model, loss_fn)`

Evaluates the model given a loss function on the entire test dataset. Returns the average accuracy (as a *float*) and the average loss over all samples (as a *float*). Sets the model to `eval` mode at the beginning of the function. 

##### Parameters
- `dataloader` (*`needle.data.DataLoader`*) - dataloader returning samples from the test dataset
- `model` (*`needle.nn.Module`*) - neural network 
- `loss_fn` (*`needle.nn.Module` type*) - loss function
___

`train_mnist(batch_size=100, epochs=10, optimizer=ndl.optim.Adam, 
                lr=0.001, weight_decay=0.001, hidden_dim=100, data_dir="data")`
                
Initializes a training dataloader (with `shuffle` set to `True`) and a test dataloader for MNIST data, and trains an `MLPResNet` using the given optimizer and the softmax loss for a given number of epochs. Returns a tuple of the training accuracy, training loss, test accuracy, test loss computed in the last epoch of training. If any parameters are not specified, use the default parameters.

##### Parameters
- `batch_size` (*int*) - batch size to use for train and test dataloader
- `epochs` (*int*) - number of epochs to train for
- `optimizer` (*`needle.optim.Optimizer` type*) - optimizer type to use
- `lr` (*float*) - learning rate 
- `weight_decay` (*float*) - weight decay
- `hidden_dim` (*int*) - hidden dim for `MLPResNet`
- `data_dir` (*int*) - directory containing MNIST image/label files


In [10]:
!python3 -m pytest -v -k "test_mlp"

============================= test session starts ==============================
platform linux -- Python 3.7.5, pytest-6.2.5, py-1.10.0, pluggy-1.0.0 -- /home/bowenc/dev/cmu/dlsys/hw2/hw2-env/bin/python3
cachedir: .pytest_cache
rootdir: /home/bowenc/dev/cmu/dlsys/hw2
collected 98 items / 88 deselected / 10 selected                               

tests/test_nn_and_optim.py::test_mlp_residual_block_num_params_1 PASSED  [ 10%]
tests/test_nn_and_optim.py::test_mlp_residual_block_num_params_2 PASSED  [ 20%]
tests/test_nn_and_optim.py::test_mlp_residual_block_forward_1 PASSED     [ 30%]
tests/test_nn_and_optim.py::test_mlp_resnet_num_params_1 PASSED          [ 40%]
tests/test_nn_and_optim.py::test_mlp_resnet_num_params_2 PASSED          [ 50%]
tests/test_nn_and_optim.py::test_mlp_resnet_forward_1 PASSED             [ 60%]
tests/test_nn_and_optim.py::test_mlp_resnet_forward_2 PASSED             [ 70%]
tests/test_nn_and_optim.py::test_mlp_train_epoch_1 PASSED                [ 80%]
tests/test

 0.25589356 0.41115716 0.         0.12285709]
<needle.nn.Dropout object at 0x7f9fc5c86550> [0.05111052 0.         0.         0.         0.10036533 0.36427844
 0.25589356 0.41115716 0.         0.12285709]
<needle.nn.Linear object at 0x7f9fc5c86050> [ 0.06809044 -0.356143    0.19565867  0.18559179  0.2821227  -0.13940634
 -0.40675783  0.08503041 -0.29686856  0.290977  ]
<needle.nn.BatchNorm object at 0x7f9fc5c86790> [ 0.0153976  -0.15082048  0.03254517  0.02525592  0.0069496  -0.08864184
  0.00237146  0.0309804  -0.14149839  0.08064843]
<mlp_resnet.ResidualBlock.<locals>.ResidualBlockModule object at 0x7f9fc5f1d510> [0.16906177 0.         0.24530736 0.24906407 0.19630827 0.
 0.00237146 0.06378569 0.         0.48618445]
<needle.nn.Linear object at 0x7f9fc6793510> [ 0.27345648 -0.20192282  0.09433682 -0.20538406  0.01955862  0.25593134
 -0.08743467  0.05152444 -0.31556409  0.03909337]
<needle.nn.BatchNorm object at 0x7f9fc6793690> [ 0.19079097 -0.4114207   0.06207205 -0.00791721  0.1567117

<needle.nn.BatchNorm object at 0x7f9fbec3f290> [-0.14697513 -0.6112874   0.806059    0.4263899  -0.1290745  -0.21278685
 -1.027003   -0.83743227  0.12104362  0.27765086]
<mlp_resnet.ResidualBlock.<locals>.ResidualBlockModule object at 0x7f9fc5f70050> [0.         0.         0.806059   0.48628616 0.         0.52346826
 0.         0.         1.2578393  0.67306805]
<needle.nn.Linear object at 0x7f9fc3a5c210> [-0.19937904  0.0193241   0.09487007 -0.00459441 -0.08425921 -0.10655317
 -0.0172594  -0.0342991   0.01722476 -0.00638119]
<needle.nn.BatchNorm object at 0x7f9fc3a5cc90> [-0.6342637  -0.18089898  1.0155596  -0.20467152  0.7680463  -1.5691109
  0.38143948 -0.01265009  2.1567852  -0.97586167]
<needle.nn.ReLU object at 0x7f9fc53e6590> [0.         0.         1.0155596  0.         0.7680463  0.
 0.38143948 0.         2.1567852  0.        ]
<needle.nn.Dropout object at 0x7f9fc53e6ed0> [0.         0.         0.         0.         0.         0.
 0.42382166 0.         2.396428   0.        ]
<ne

 0.         0.         0.6346113  0.        ]
<needle.nn.Dropout object at 0x7f9fbec3f250> [0.         0.         0.         0.59458745 0.         0.
 0.         0.         0.6346113  0.        ]
<needle.nn.Linear object at 0x7f9fbec3f350> [-0.05038208 -0.14815497 -0.22027688  0.2804065  -0.11765376  0.00813136
  0.2681032   0.5266333  -0.28514084  0.27857512]
<needle.nn.BatchNorm object at 0x7f9fbec3f290> [ 0.5272257  -0.26204878  0.22069213  0.90710574 -0.46212202  0.674482
  0.8168886   1.5587025  -1.0442802   1.0316576 ]
<mlp_resnet.ResidualBlock.<locals>.ResidualBlockModule object at 0x7f9fc5f70050> [1.6481004  0.         0.75224996 3.1406186  0.         0.674482
 1.2725296  3.9577172  0.         2.826239  ]
<needle.nn.Linear object at 0x7f9fc3a5c210> [-0.01886234  0.22510572  0.12159905 -0.01178644 -0.06647976  0.05383278
 -0.13014593 -0.04525918 -0.16585809 -0.03902856]
<needle.nn.BatchNorm object at 0x7f9fc3a5cc90> [ 2.1504803   2.6735048   1.158778   -0.5359618   1.0697567   1

In [14]:
!python3 -m mugrade submit 'eAzORHnc5JIEkr0dqEig' -k "mlp_resnet"

submit
============================= test session starts ==============================
platform linux -- Python 3.7.5, pytest-6.2.5, py-1.10.0, pluggy-1.0.0
rootdir: /home/bowenc/dev/cmu/dlsys/hw2
collected 22 items / 21 deselected / 1 selected                                

tests/test_nn_and_optim.py 
Submitting mlp_resnet...
Grader test 1 passed
Grader test 2 passed
Grader test 3 passed
Grader test 4 passed
Grader test 5 passed
Grader test 6 passed
Grader test 7 passed
0 [ 0.03335927  0.00337373  0.03376317  0.015344    0.01412349 -0.02027932
  0.0340196  -0.03526927 -0.01764412 -0.00465775]
0 [ 3.33592732e-04  3.37373203e-05  3.37631704e-04  1.53439993e-04
  1.41234865e-04 -2.02793221e-04  3.40196042e-04 -3.52692674e-04
 -1.76441157e-04 -4.65774756e-05]
0 [1.11284111e-07 1.13820678e-09 1.13995167e-07 2.35438313e-08
 1.99472870e-08 4.11250903e-08 1.15733347e-07 1.24392122e-07
 3.11314820e-08 2.16946123e-09]

1 [-0.02770682 -0.01399836 -0.02728     0.02651393  0.0001961  -0.0175064

0 [ 0.01360234 -0.00303266  0.01400085 -0.00343079 -0.0043935  -0.00093857
  0.01425397 -0.01548855  0.00146209  0.00413437]
0 [ 2.27479592e-04 -2.31029866e-05  2.31498676e-04  5.09521597e-05
  3.96985180e-05 -9.82164352e-05  2.34050773e-04 -2.46491687e-04
 -7.27301125e-05  1.90689740e-05]
0 [6.14202086e-08 2.14816842e-09 6.33244516e-08 9.18500603e-09
 7.85348280e-09 1.72465285e-08 6.45503358e-08 7.07120638e-08
 1.23861470e-08 2.24332564e-09]

1 [-0.03279508  0.00581537 -0.0232395   0.03035743 -0.0108789  -0.03065407
  0.00266401]
1 [-0.03417329 -0.0274379   0.05481998  0.02422543 -0.01530065  0.02190046
  0.01156219]
1 [0.00249672 0.00123241 0.02308761 0.00184913 0.01477818 0.00686762
 0.00148858]

2 [ 0.3331508   0.03007462  0.37571388  0.14831616  0.1638975  -0.23394555
  0.34003207 -0.37722802 -0.16702783 -0.04522751]
2 [ 0.01924836 -0.02341653 -0.00985029  0.03503524 -0.00464339  0.01354297
  0.01974856 -0.01378021 -0.03665539  0.00348723]
2 [0.00049493 0.00117986 0.00017459 0.006

0 [-0.00365111  0.00454408 -0.00332511 -0.0098874  -0.00913825  0.01081936
 -0.00311651  0.00208322  0.01071143 -0.00326503]
0 [ 1.22742883e-04  3.00007627e-06  1.26523741e-04 -1.60555787e-05
 -2.05352786e-05 -1.18542147e-05  1.28929574e-04 -1.40708743e-04
  4.74543257e-06  2.48842469e-06]
0 [3.74742027e-08 1.77760834e-09 3.86398180e-08 8.99624161e-09
 7.95691869e-09 1.35396227e-08 3.93950684e-08 4.32461483e-08
 1.10037026e-08 1.55852280e-09]

1 [-0.01846231  0.01952089 -0.02911259  0.01945345 -0.00789715 -0.03494567
 -0.00233372]
1 [-0.03636187 -0.01570695  0.01543045  0.01779563 -0.02182414  0.0032268
  0.00736449]
1 [0.00210246 0.00075921 0.01420172 0.00115162 0.00920426 0.00425918
 0.0009153 ]

2 [ 0.31566727  0.04444134  0.38611695  0.1412573   0.16540474 -0.24348289
  0.32981834 -0.36834738 -0.15347238 -0.04921478]
2 [ 0.04457758 -0.02116563 -0.00441398  0.00714697  0.00231915  0.00587877
  0.00893702 -0.00901058 -0.01728221  0.00388801]
2 [0.00255776 0.00088871 0.00013781 0.0043

25 [ 0.01796836  0.04270131 -0.04339961 -0.04382476  0.02175819 -0.04800684
 -0.03323436]
25 [-0.02572086 -0.00794178  0.01553867  0.0117895   0.00093384  0.04810236
  0.00379065]
25 [8.5711631e-04 1.9946862e-04 7.5031637e-04 3.6598474e-04 2.9313225e-05
 3.1837844e-03 1.1906485e-04]

26 [ 0.10706903 -0.21886793 -0.02615917 -0.23989865  0.3644492  -0.10043927
  0.2646228  -0.08200005  0.26149562 -0.07286379]
26 [-0.00140673  0.02765479  0.02103797  0.01295968 -0.08499622  0.03716321
 -0.06532162  0.0371691   0.05140214 -0.03274197]
26 [0.00368254 0.00224765 0.00141211 0.00028808 0.01641452 0.00176881
 0.00578507 0.00226966 0.00301322 0.00161162]

27 [ 0.19968314  0.31486315  0.30883294 -0.33467564 -0.31601766  0.00513987
 -0.30357495  0.07423593  0.04500782  0.17269474]
27 [ 0.00391517  0.03343971  0.04206379  0.00356748 -0.04405069 -0.0033161
 -0.02361346  0.03350482 -0.00982271 -0.03409304]
27 [0.00031743 0.00130552 0.00183408 0.00037463 0.00279145 0.00048916
 0.00063646 0.00224329 0.


18 [ 0.3165957   0.02049353  0.37854612  0.12851742  0.08898903 -0.19303013
  0.37959525 -0.32782173 -0.14080837 -0.10006461]
18 [ 0.05941876  0.00455534 -0.00450694  0.03101771  0.01270004 -0.02295161
 -0.02668511 -0.00411989 -0.00264394  0.01226429]
18 [0.00506932 0.00030423 0.00046644 0.00131756 0.0002672  0.00101189
 0.00119674 0.00058377 0.00037168 0.00021994]

19 [0.26981258 0.14716478 0.20742537]
19 [0.00299081 0.00176227 0.00236617]
19 [9.3216277e-06 3.3487543e-06 5.9076592e-06]

20 [0.93684083 0.9599533  0.9985689 ]
20 [0.01864634 0.0034082  0.00694325]
20 [0.00055815 0.00015783 0.00017571]

21 [-0.05564324 -0.01951156  0.04319455]
21 [ 0.00048331 -0.01094939 -0.00436092]
21 [5.3514934e-05 6.6013238e-04 4.9596751e-05]

22 [-0.31936044 -0.48333612  0.12856975 -0.36158416  0.20859554 -0.11304392
  0.07222298  0.5331202   0.01231421 -0.41595468]
22 [-0.02848723 -0.00643939 -0.0073948  -0.00242086  0.01711325  0.00949341
 -0.03824266 -0.0163454  -0.00837765 -0.00696681]
22 [1.587

...

 -0.02487562  0.12479328 -0.01669601 -0.11866497]
22 [-0.00238997 -0.00181296 -0.00636266  0.00025702  0.0006332  -0.00274516
  0.00150432  0.00237995 -0.0025957  -0.00924742]
22 [5.8120029e-04 2.2982984e-04 9.5927349e-04 8.7689841e-05 1.9132889e-04
 1.9366886e-04 1.4543898e-03 4.6014931e-04 2.8265672e-04 6.8525807e-04]

23 [ 2.9098452e-05  2.5657823e-07  5.9737607e-07 -5.1980551e-06
  1.6027504e-07  4.4061399e-06  2.8408176e-06]
23 [-4.9732289e-09 -4.1175374e-10 -2.9726227e-08 -2.0621380e-09
  2.2289020e-09 -2.5174577e-08 -1.6515568e-08]
23 [7.7409912e-11 1.4973185e-07 3.3622638e-07 6.1717351e-09 4.8859963e-08
 1.3971822e-06 6.2983396e-08]

24 [0.43571725 0.3173895  0.5197957  0.36375824 0.36500812 0.22312179
 0.5452747 ]
24 [-0.00030804 -0.00174708  0.00052264 -0.00434587 -0.00115293  0.00038536
 -0.00158016]
24 [1.5172818e-04 9.6748350e-05 1.1522678e-04 1.0900506e-04 9.5768541e-05
 1.0309650e-04 1.2472786e-04]

25 [ 0.08773062  0.03063969  0.10911971 -0.0062869   0.21863139  0.13906

27 [-0.03595946  0.23262879  0.0370783  -0.13874601 -0.05888583  0.3383344
 -0.13854994  0.07412865 -0.23666738  0.01704205]
27 [-2.4255121e-04 -1.6378541e-03  2.0550115e-03 -1.8292690e-03
  9.0721587e-05 -1.9928000e-03 -2.5043653e-03 -1.8405821e-04
  4.3663653e-03  2.8051487e-03]
27 [0.00018187 0.00020869 0.00034052 0.000351   0.00035521 0.0003923
 0.00023673 0.00022028 0.00030502 0.00030411]

0 [-1.8284088e-07  4.3423920e-08 -1.2921987e-07  8.8366200e-08
  7.7830073e-08 -9.6998470e-08 -6.6402841e-08 -1.9620830e-07
  9.9932720e-08  3.9629047e-09]
0 [-6.72891303e-11  1.75833862e-11  1.62997909e-10  6.85961380e-11
  1.11525754e-11 -1.45702335e-10  2.83585732e-10 -1.51973213e-10
  8.03571804e-11 -2.56104745e-11]
0 [1.48758223e-09 6.91192313e-11 1.52569954e-09 3.24488732e-10
 2.79681574e-10 5.46136483e-10 1.55020180e-09 1.67299243e-09
 4.19509260e-10 5.98491557e-11]

1 [0.18060625 0.14076994 0.168682   0.0002781  0.17782982 0.13977651
 0.04682615]
1 [ 0.00197189 -0.00082385 -0.00049162 -0

0 [-1.0668641e-07 -2.3613657e-08 -1.6471955e-07 -1.2009059e-08
  2.2520924e-08  3.6415308e-08 -1.6814477e-07 -8.6575930e-08
 -4.3623474e-09  1.8939456e-08]
0 [-3.10049745e-10  1.06986045e-11 -1.86536430e-10  8.88301754e-11
  9.56033000e-11 -1.12813859e-10 -6.30099568e-11 -3.52830169e-10
  1.15373751e-10  3.15172787e-11]
0 [1.47322676e-09 6.84522168e-11 1.51097623e-09 3.21357350e-10
 2.76982590e-10 5.40866155e-10 1.53524204e-09 1.65684772e-09
 4.15460911e-10 5.92715993e-11]

1 [0.1791089  0.14341451 0.16901477 0.00790732 0.18051685 0.14227779
 0.03998262]
1 [ 0.00146358 -0.00641956 -0.00150051 -0.00157601 -0.00403235 -0.00036988
  0.01005958]
1 [0.00050564 0.00024145 0.00057942 0.00022992 0.00040443 0.00019598
 0.00042786]

2 [ 0.0096774   0.01740577  0.1794224  -0.08651754  0.05039815 -0.04572367
  0.1027256  -0.36006513 -0.19445327 -0.23750241]
2 [ 0.00285319 -0.0114637   0.01231677  0.00088415  0.00771471 -0.00119441
  0.00810739 -0.00145794  0.00099501  0.00614598]
2 [0.00147287 0.0

0 [ 5.3698283e-08 -3.9765404e-09 -4.3363052e-08 -7.1824552e-08
 -6.5680972e-08  9.2067580e-08 -9.8669098e-08  7.8245229e-08
 -8.2024677e-08 -3.0293830e-08]
0 [-2.20777929e-10 -2.81868449e-11 -2.99526843e-10 -5.52527299e-11
 -1.96422564e-11  7.71009863e-11 -2.86567983e-10 -2.07471718e-10
 -4.34160570e-11 -2.45156226e-11]
0 [1.45911777e-09 6.77966548e-11 1.49650572e-09 3.18279734e-10
 2.74329948e-10 5.35686319e-10 1.52053914e-09 1.64098021e-09
 4.11482072e-10 5.87039595e-11]

1 [0.17833574 0.15087493 0.17054845 0.00918173 0.18508002 0.14272125
 0.02996998]
1 [ 8.4929948e-04 -2.2963621e-03 -2.2077421e-03  9.0033148e-04
 -6.6030188e-03 -4.1544597e-05  7.4711386e-03]
1 [0.00050149 0.00024196 0.00057424 0.00022897 0.00040352 0.00019412
 0.0004271 ]

2 [ 0.00839184  0.02293143  0.16842113 -0.08706003  0.04583358 -0.04557609
  0.0963488  -0.35917917 -0.19505483 -0.24161722]
2 [ 0.00331774 -0.00724307  0.01338773 -0.0031071   0.00866287  0.0017576
  0.00565088 -0.00126428  0.00228129  0.0034012

Grader test 8 passed
Grader test 9 passed
train_acc 0.29 train_loss 2.02 test_acc 0.41 test_loss 1.75
train_acc 0.42 train_loss 1.73 test_acc 0.52 test_loss 1.55
train_acc 0.51 train_loss 1.57 test_acc 0.63 test_loss 1.40
Grader test 10 passed
.

================= 1 passed, 21 deselected in 70.34s (0:01:10) ==================


We encourage to experiment with the `mlp_resnet.py` training script.
You can uncomment the lines that print training statistics and investigate
the effect of using different initializers on the Linear layers,
increasing the dropout probability,
or adding transforms (via a list to the `transforms=` keyword argument of Dataset)
such as random cropping.

In [42]:
import sys
sys.path.append('./python')
from apps import mlp_resnet